In [ ]:
from spylus.utils import *
import ipycanvas as ca
from ipyfilechooser import FileChooser
import numpy as np
from PIL import Image

In [ ]:
from contextlib import redirect_stdout
import os
import torch
from util import util
from models.pix2pix_model import Pix2PixModel
from options.test_options import TestOptions

with redirect_stdout(open(os.devnull, 'w')):
    opt = TestOptions().parse(
    """
    --conf ./parameters/migan_leak_4_128.yml
    --which_epoch 1
    """.split())
    model = Pix2PixModel(opt)
    model.eval()

In [ ]:
ID, canvas = ID_multicanvas()
marker = HTML(f'<div id="{ID}"></div>')
selector = wi.Dropdown(options=range(2), value=1)

save_input0 = wi.Text(layout=wi.Layout(width="30px"))
save_input1 = wi.Text(layout=wi.Layout(width="30px"))
copy_button0 = wi.Button(description="copy", layout=wi.Layout(width="130px"))
copy_button1 = wi.Button(description="copy", layout=wi.Layout(width="130px"))
paste_button0 = wi.Button(description="paste", layout=wi.Layout(width="130px"))
paste_button1 = wi.Button(description="paste", layout=wi.Layout(width="130px"))
reload_button0 = wi.Button(description="reload", layout=wi.Layout(width="130px"))
clear_button1 = wi.Button(description="clear", layout=wi.Layout(width="130px"))
color_input0 = wi.ColorPicker(value='black', layout=wi.Layout(width="130px"))
color_input1 = wi.ColorPicker(value='#ff8000', layout=wi.Layout(width="130px"))
width_select0 = wi.Dropdown(options=range(1,20), value=2, layout=wi.Layout(width="130px"))
width_select1 = wi.Dropdown(options=range(1,20), value=2, layout=wi.Layout(width="130px"))

out = wi.Output()
fc0 = FileChooser("/home/natsuki/sketch_simplification", use_dir_icons=True)
@out.capture(clear_output=True)
def load0(chooser):
    image = Image.open(fc0.selected)
    text = encode(image)
    display(Javascript(f'app0.load("{text}");'))
fc0.register_callback(load0)

out_ca = ca.Canvas(width=512, height=512)
out_ca.sync_image_data = True
out_ca.fill_style = '#a9cafc'
out_ca.fill_rect(0, 0, 512, 512)
out_ca.fill_style = 'black'
out_ca.font = '32px serif'
out_ca.fill_text('Initialized', 0, 512)

synth_button = wi.Button(description="synthesize", layout=wi.Layout(width="100px"))
@out.capture(clear_output=True)
def synth(b, filter=Image.BICUBIC):
    out_ca.fill_text('Synthesizing...', 0, 512)
    sketch_pillow = decode(save_input0.value).resize((256, 256), filter)
    sketch_numpy =  np.array(sketch_pillow, dtype='float32')
    sketch_torch = torch.from_numpy(1 - sketch_numpy.transpose((2,0,1))[None,:1,:,:]/255).cuda() #Rのみ
    hint_pillow = decode(save_input1.value).resize((256, 256), filter)
    hint_numpy =  np.array(hint_pillow, dtype='float32')
    hint_torch = torch.from_numpy(hint_numpy.transpose(2,0,1)[None,:3,:,:]/127.5-1).cuda()
    input_torch = torch.cat((sketch_torch,hint_torch),dim=1)
    output_torch = model.netG(input_torch)
    output_torch = output_torch[0]
    output_numpy = util.tensor2im(output_torch)
    output_pillow = Image.fromarray(output_numpy).resize((512, 512), filter)
    output_numpy = np.array(output_pillow)
    out_ca.put_image_data(output_numpy)
synth_button.on_click(synth)

@out.capture(clear_output=True)
def on_mouse_down(x, y):
    color_input1.value = '#'+''.join(map(lambda c: hex(c)[2:], out_ca.get_image_data()[int(y),int(x)][0:3]))
out_ca.on_mouse_down(on_mouse_down)

reload_button0.on_click(load0)

app =  wi.VBox((
    wi.HBox((out_ca, canvas,)),
    wi.HBox((save_input0, copy_button0, paste_button0, reload_button0, color_input0, width_select0,)),
    wi.HBox((save_input1, copy_button1, paste_button1, clear_button1, color_input1, width_select1,)),
    wi.HBox((selector, synth_button)),
    fc0,
    out,
))
display(marker, app)

jscode = Javascript(f"""
{Path('spylus/spylus.js').read_text()}
window.app0 = new (mix(
    Save, Load, Copy, Paste, Color, Width
))({{
    canvas: document.getElementById("{ID}0"),
    save_input: elem('{ID}', 'input', 0),
    copy_button: elem('{ID}', 'button', 0),
    paste_button: elem('{ID}', 'button', 1),
    color_input: elem('{ID}', 'input', 1),
    width_select: elem('{ID}', 'select', 0),
}});

window.app1 = new (mix(
    White, Save, Copy, Paste, Color, Width, Clear
))({{
    canvas: document.getElementById("{ID}1"),
    save_input: elem('{ID}', 'input', 3),
    copy_button: elem('{ID}', 'button', 3),
    paste_button: elem('{ID}', 'button', 4),
    clear_button: elem('{ID}', 'button', 5),
    color_input: elem('{ID}', 'input', 4),
    width_select: elem('{ID}', 'select', 1),
}});

new Selector({{
    apps: [app0, app1],
    selector_select: elem('{ID}', 'select', 2),
}});
""")
with out:
    display(jscode)